<a href="https://colab.research.google.com/github/lauramenicacci/patent_breakdetection/blob/main/REGPAT_extract_patent_counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data collection from OECD Patent Database

Collect disaggregated patent raw data from OECD STI [Micro-data Lab](https://www.oecd.org/sti/intellectual-property-statistics-and-analysis.htmOECD/STI). 

* OECD REGPAT DATABASE - EPO & PCT Patent Applications at Regional Level

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Import libraries 

In [ ]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


 ## REGPAT - EPO 
 
Collect regional patent data filed at the EPO.
 - State level 
 - TL2 regional classification (CITE OECD) 
 - 51 regions 

In [ ]:
regpat_regions = pd.read_csv('/content/REGPAT_REGIONS.txt', encoding = 'utf-8', sep = "|") # table with description of regional codes 

In [ ]:
epo_inventors_filt = pd.read_csv('/content/drive/MyDrive/epo_inventors_filt.csv', encoding = 'utf-8', sep = ',') # List of EPO inventors: contains appl_id

In [ ]:
cpc_class = pd.read_csv("/content/drive/MyDrive/202208_CPC_Classes.txt",  encoding = "utf-8", sep = "|") # list of CPC classes related to the invention

In [ ]:
cpc_y02e = pd.read_csv('/content/cpc_y02e.csv') # cpc technology classification

In [ ]:
epo_ipc = pd.read_csv("/content/drive/MyDrive/202208_EPO_IPC.txt", encoding = "utf-8", sep = "|") 

### Filter by Y02E class

In [ ]:
cpc_y02e = cpc_class[cpc_class['CPC_Class'].str.startswith('Y02E')] # select class that starts with Y02E

In [ ]:
len(cpc_y02e)

352222

In [ ]:
cpc_y02e.to_csv('cpc_y02e.csv', index = False)

### Join with list of IPC classes and CPC classes

In [ ]:
merged_ipc_cpc = pd.merge(epo_ipc, cpc_y02e, how='left', on= 'appln_id') # merge epo patents with patent class dataset

merged_ipc_cpc = merged_ipc_cpc.dropna()

len(merged_ipc_cpc)

### Merge regional US data with years and cpc class 

In [ ]:
# merge regpat regions and epo inventors data using ctry_code & reg_code

regpat_regions.rename(columns = {'Ctry_code':'ctry_code'}, inplace = True) # rename col

merged1 = pd.merge(epo_inventors_filt, regpat_regions, how='left', on=['ctry_code', 'reg_code'])

merged1 = merged1.drop_duplicates()

merged1.head() # we have now application id and TL2 regions in the same df

,app_nbr,appln_id,reg_code,ctry_code,reg_share,inv_share,reg_label,Up_reg_code,Up_reg_label
0,EP20000103094,1,FI197,FI,1.0,0.250000,Pirkanmaa,FI19,Western Finland
4,EP20000107845,2,UKH12,GB,1.0,0.166667,Cambridgeshire CC,UKH,East of England
6,EP20000107845,2,US06001,US,1.0,0.166667,"Alameda County, CA",US06,California
9,EP20000107845,2,AU1GS,AU,1.0,0.166667,Sydney,AU1,New South Wales
10,EP20000202556,3,DEA2D,DE,1.0,0.333333,Aachen region,DEA2,Köln


In [ ]:
merged1_us = merged1.loc[merged1['ctry_code'] == 'US', ] # filter for US only

merged2 = pd.merge(merged1_us, merged_ipc_cpc, how='left', on= 'appln_id') # merge Y02E class & year data with US regional data 

regpat_epo = merged2.dropna() # we drop NAs because they are only present in time and CPC class = means that those invetions were not Y02E

regpat_epo = regpat_epo.astype({'prio_year':'int', 'app_year':'int'})

regpat_epo.head()

,app_nbr,appln_id,reg_code,ctry_code,reg_share,inv_share,reg_label,Up_reg_code,Up_reg_label,prio_year,app_year,IPC,CPC_Class
95,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,B62B003/14,Y02E 60/10
96,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K007/00,Y02E 60/10
97,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K007/08,Y02E 60/10
98,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K007/10,Y02E 60/10
99,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K017/00,Y02E 60/10


In [ ]:
len(regpat_epo.loc[:,'appln_id'].unique()) # number of US inventions under Y02E class filed at EPO

27577

In [ ]:
regpat_epo.to_csv('epo_regpat_agg.csv', sep = ',', index = False) # save

### Create patent counts

In [ ]:
epo = pd.read_csv("/content/epo_regpat_agg.csv")

In [ ]:
epo_filt = epo[['Up_reg_label', 'prio_year','inv_share', 'appln_id']].drop_duplicates() # filter by State, priority year, share of patent and application id

In [ ]:
epo_count = epo_filt.groupby(['Up_reg_label', 'prio_year'], as_index = False).inv_share.sum() # create count by summing over inventor share

epo_count.head()

In [ ]:
epo_count.to_csv('epo_patent_counts.csv', sep = ",", index = False)

## REGPAT - PCT

Collect regional patent data filed at the EPO.
 - State level 
 - TL2 regional classification (CITE OECD) 
 - 51 regions 

In [ ]:
pct_inv_filt = pd.read_csv("/content/drive/My Drive/pct_inv_filt.csv", encoding = "utf-8", sep = ",")

pct_inv_us = pct_inv_filt.loc[pct_inv_filt['ctry_code'] == 'US', ].drop_duplicates()

In [ ]:
pct_ipc = pd.read_csv("/content/drive/MyDrive/202208_PCT_IPC.txt", encoding = "utf-8", sep = "|")

In [ ]:
merged_ipc_inv = pd.merge(pct_inv_us, pct_ipc, how='left', on= 'pct_nbr') # merge pct_ipc with pct inventors data: we have country and year information

merged_ipc_inv.head()

In [ ]:
merged_ipc_inv.isna().value_counts() # 13 nas for appln_id

pct_nbr  appln_id  reg_code  ctry_code  reg_share  inv_share  prio_year  app_year  IPC  
False    False     False     False      False      False      False      False     False    8445469
                                                                                   True        1137
         True      False     False      False      False      False      False     False         94
                                                                                   True           1
dtype: int64

In [ ]:
merged_ipc_inv = merged_ipc_inv.dropna()

merged_ipc_inv = merged_ipc_inv.astype({'appln_id':'int'}) # transform appln_id from numerical to integer for merging 

print(merged_ipc_inv.dtypes)

In [ ]:
merged_ipc_y02e = pd.merge(merged_ipc_inv, cpc_y02e, how='left', on= 'appln_id') # merge ipc_inv with y02e CPC class 

merged_ipc_y02e = merged_ipc_y02e.dropna()

In [ ]:
regpat_regions.rename(columns = {'Ctry_code':'ctry_code'}, inplace = True) # rename col

reg_pct = pd.merge(merged_ipc_y02e, regpat_regions, how='left', on=['ctry_code', 'reg_code']).drop_duplicates() # merge geographical information with patent information

In [ ]:
len(reg_pct['appln_id'].unique()) # number of patent applications at PCT  

36313

In [ ]:
reg_pct.isna().value_counts() # sanity check

In [ ]:
reg_pct.to_csv('pct_aggregated.csv', sep = ",", index = False) # csv with all aggregated pct data

### Create patent counts

In [ ]:
reg_pct_filt = reg_pct[['Up_reg_label', 'prio_year','inv_share', 'appln_id']].drop_duplicates() # select columns of interest

In [ ]:
pct_counts = reg_pct_filt.groupby(['Up_reg_label', 'prio_year', 'appln_id'], as_index = False).inv_share.sum() # create patent counts: sum over inventor share - keeping application id

In [ ]:
pct_counts_2 = pct_counts.groupby(['Up_reg_label','prio_year'], as_index = False).inv_share.sum() # create patent counts: sum over inventor share

In [ ]:
pct_counts_2.to_csv('pct_patent_counts.csv', index = False, sep = ",") # save

## EPO-PCT overlap 

Check whether there are overlapping patents: in case yes, we have to remove them to avoid double counting!

In [ ]:
epo_pct = pd.read_csv("/content/202208_EPO_PCT.txt", encoding = "utf-8", sep = "|")

In [ ]:
epo = pd.read_csv("/content/epo_regpat_agg.csv")
epo.head()

,Unnamed: 0,app_nbr,appln_id,reg_code,ctry_code,reg_share,inv_share,reg_label,Up_reg_code,Up_reg_label,prio_year,app_year,IPC,CPC_Class
0,95,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,B62B003/14,Y02E 60/10
1,96,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K007/00,Y02E 60/10
2,97,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K007/08,Y02E 60/10
3,98,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K007/10,Y02E 60/10
4,99,EP20020022517,1297,US48339,US,1.0,0.5,"Montgomery County, TX",US48,Texas,1996,1997,G06K017/00,Y02E 60/10


In [ ]:
len(epo['app_nbr'].unique())  

27577

In [ ]:
# check intersection between EPO patents filtered for US and PCT patents that entered EPO regional phase 

A = set(epo['app_nbr']) 
B = set(epo_pct['app_nbr'])
print(len(A.intersection(B)))
pct_epo_common = A.intersection(B) # 18313 

18313


## Check whether filing date is the same

For the overlapping patents: is priority filing the same? This is a test for checking quality of the data. If it is the same, we can merge without problems. 

In [ ]:
common_epo = epo[epo['app_nbr'].isin(pct_epo_common)]

In [ ]:
print(len(common_epo['appln_id'].unique()))

print(len(common_epo))

18313
210918


In [ ]:
# For the patents which have both an EPO and a PCT application, is the filing date the same? 

common_con_pctnbr = pd.merge(common_epo, epo_pct, how = 'left', on = ['app_nbr'])

print(len(common_con_pctnbr))

common_con_pctnbr.head()

210918


,Unnamed: 0,app_nbr,appln_id,reg_code,ctry_code,reg_share,inv_share,reg_label,Up_reg_code,Up_reg_label,prio_year,app_year,IPC,CPC_Class,pct_nbr
0,147,EP20020795916,1357,US18097,US,1.0,0.25,"Marion County, IN",US18,Indiana,2002,2002,B01D053/94,Y02E 60/50,WO2003064833
1,148,EP20020795916,1357,US18097,US,1.0,0.25,"Marion County, IN",US18,Indiana,2002,2002,B01J019/08,Y02E 60/50,WO2003064833
2,149,EP20020795916,1357,US18097,US,1.0,0.25,"Marion County, IN",US18,Indiana,2002,2002,C01B003/34,Y02E 60/50,WO2003064833
3,150,EP20020795916,1357,US18097,US,1.0,0.25,"Marion County, IN",US18,Indiana,2002,2002,F01N003/02,Y02E 60/50,WO2003064833
4,151,EP20020795916,1357,US18097,US,1.0,0.25,"Marion County, IN",US18,Indiana,2002,2002,F01N003/025,Y02E 60/50,WO2003064833


In [ ]:
common_epo_filt = common_con_pctnbr[['prio_year', 'pct_nbr']].drop_duplicates()

print(common_epo_filt.head())

pct_filt = pct[['prio_year', 'pct_nbr']].drop_duplicates()

print(pct_filt.head())

     prio_year       pct_nbr
0         2002  WO2003064833
51        2002  WO2004013868
59        2002  WO2004051774
71        2004  WO2005083829
139       2004  WO2005109551
    prio_year       pct_nbr
0        1977  WO1978000019
2        1977  WO1979000018
12       1977  WO1979000028
13       1977  WO1979000030
17       1977  WO1979000140


In [ ]:
# find equal pct_nbr between the two dfs and check if the prio_year is the same 

tupla1 = zip(common_con_pctnbr['prio_year'], common_con_pctnbr['pct_nbr'])
tupla2 = zip(pct_filt['prio_year'], pct_filt['pct_nbr'])

In [ ]:
A = set(tupla1) 
B = set(tupla2)
print(len(A.intersection(B)))
epo_pct_year_same = A.intersection(B) # 100 patents out of 18.000 don't have the same priority year: good result!

18214


## Remove overlapping patents in the EPO data 

In [ ]:
if ('EP20020022517' in pct_epo_common):  # test
      print("Element Exists")
else:
        print("noooo")

noooo


In [ ]:
epo_extra = epo[~epo['app_nbr'].isin(pct_epo_common)] # drop matching application numbers from epo aggregated regpat 

In [ ]:
print(len(epo_extra)) 

len(epo_extra['app_nbr'].unique()) # 9264 patents are direct applications from US to EPO

87904


9264

## Concatenate direct EPO patents with PCT patents

In [ ]:
pct = pd.read_csv("/content/pct_aggregated.csv")

In [ ]:
len(pct['appln_id'].unique())

36313

In [ ]:
filt_pct = pct[['Up_reg_label', 'prio_year','inv_share', 'appln_id']].drop_duplicates()

print(filt_pct.head())

filt_epo = epo_extra[['Up_reg_label', 'prio_year','inv_share', 'appln_id']].drop_duplicates()

print(filt_epo.head())

In [ ]:
# merge pct patent counts and patent counts from direct applications from us to epo 

pct_counts = filt_pct.groupby(['Up_reg_label','prio_year', 'appln_id'], as_index = False).inv_share.sum() # create pct counts 

print(pct_counts.head())

direct_epo_counts = filt_epo.groupby(['Up_reg_label','prio_year', 'appln_id'], as_index = False).inv_share.sum() # create direct epo counts

print(direct_epo_counts.head())

In [ ]:
print(len(pct_counts))
print(len(direct_epo_counts))

45519
11427


In [ ]:
# concatenate the two dataframes 

pct_direct_epo_all = pd.concat([pct_counts,direct_epo_counts], ignore_index= True, axis = 0)

pct_direct_epo_all.head()

In [ ]:
len(pct_direct_epo_all)

print(len(pct_direct_epo_all['appln_id'].unique())) # number of unique patent ids = n of patent applications for single inventions

45577


In [ ]:
pct_direct_epo_all.to_csv('patent_count_pct_direct_epo.csv', index = False, sep = ",") # save official final dataset to be used for analysis

## Patent count without appln_id

In [ ]:
pct_direct_epo_all_counts = pct_direct_epo_all.groupby(['Up_reg_label','prio_year'], as_index = False).inv_share.sum()

pct_direct_epo_all_counts.head()

pct_direct_epo_all_counts.to_csv('patent_count_pct_direct_epo_counts.csv', index = False, sep = ",")

# For graphs

In [ ]:
pct.rename(columns={'pct_nbr': 'app_nbr'}, inplace=True)

In [ ]:
# create dataset with all information to create graphs

epo_pct_pat = pd.concat([pct, epo_extra], ignore_index=True)

epo_pct_pat.tail()

,app_nbr,appln_id,reg_code,ctry_code,reg_share,inv_share,prio_year,app_year,IPC,CPC_Class,reg_label,Up_reg_code,Up_reg_label
422684,EP20210188491,555010709,US06085,US,1.0,0.125,2015,2016,H04R025/02,Y02E 60/10,"Santa Clara County, CA",US06,California
422685,EP20210188491,555010709,US25025,US,1.0,0.125,2015,2016,A61F002/00,Y02E 60/10,"Suffolk County, MA",US25,Massachusetts
422686,EP20210188491,555010709,US25025,US,1.0,0.125,2015,2016,A61F002/18,Y02E 60/10,"Suffolk County, MA",US25,Massachusetts
422687,EP20210188491,555010709,US25025,US,1.0,0.125,2015,2016,H04R025/00,Y02E 60/10,"Suffolk County, MA",US25,Massachusetts
422688,EP20210188491,555010709,US25025,US,1.0,0.125,2015,2016,H04R025/02,Y02E 60/10,"Suffolk County, MA",US25,Massachusetts


In [ ]:
len(epo_pct_pat)

422689

In [ ]:
epo_pct_pat.to_csv('pct_direct_epo_all_vars.csv', index = False, sep = ",")